# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities.head()



,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,São João da Barra,90,BR,1612538589,69,-21.6403,-41.0511,82.40,10.20
1,Mahébourg,40,MU,1612538589,69,-20.4081,57.7000,82.40,13.80
2,Zhigalovo,100,RU,1612538589,91,54.8097,105.1578,4.44,3.58
3,Hilo,1,US,1612538539,67,19.7297,-155.0900,63.00,6.91
4,Kaitangata,95,NZ,1612538589,92,-46.2817,169.8464,46.51,5.23


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = cities["Humidity"]


In [11]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig




Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city = cities.loc[(cities["Wind Speed"] <= 10) & (cities["Cloudiness"] == 0) &
                                   (cities["Max Temp"] >= 70) & (cities["Max Temp"] <= 80)].dropna()

narrowed_city

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
15,Mar del Plata,0,AR,1612538501,60,-38.0023,-57.5575,73.40,6.91
40,Hermanus,0,ZA,1612538595,69,-34.4187,19.2345,79.00,4.00
55,Castro,0,BR,1612538596,57,-24.7911,-50.0119,73.08,7.65
171,Champasak,0,LA,1612538610,57,14.7500,106.0000,71.62,0.87
258,Upleta,0,IN,1612538620,53,21.7333,70.2833,73.40,3.44
295,Vaijāpur,0,IN,1612538625,49,19.9167,74.7333,71.60,4.72
319,Coyhaique,0,CL,1612538628,49,-45.5752,-72.0662,71.60,3.44
339,Pathein,0,MM,1612538434,86,16.7833,94.7333,70.95,4.52
358,Awjilah,0,LY,1612538634,28,29.1081,21.2869,71.02,6.62
399,Ormara,0,PK,1612538639,58,25.2088,64.6357,70.54,9.13


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# Create a hotel_df
hotel_df = narrowed_city.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
15,Mar del Plata,AR,-38.0023,-57.5575,
40,Hermanus,ZA,-34.4187,19.2345,
55,Castro,BR,-24.7911,-50.0119,
171,Champasak,LA,14.7500,106.0000,
258,Upleta,IN,21.7333,70.2833,
295,Vaijāpur,IN,19.9167,74.7333,
319,Coyhaique,CL,-45.5752,-72.0662,
339,Pathein,MM,16.7833,94.7333,
358,Awjilah,LY,29.1081,21.2869,
399,Ormara,PK,25.2088,64.6357,


In [21]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # results
    results = response['results']

hotel_df




Retrieving Results for Index 15: Mar del Plata.
Retrieving Results for Index 40: Hermanus.
Retrieving Results for Index 55: Castro.
Retrieving Results for Index 171: Champasak.
Retrieving Results for Index 258: Upleta.
Retrieving Results for Index 295: Vaijāpur.
Retrieving Results for Index 319: Coyhaique.
Retrieving Results for Index 339: Pathein.
Retrieving Results for Index 358: Awjilah.
Retrieving Results for Index 399: Ormara.
Retrieving Results for Index 426: Bhatkal.
Retrieving Results for Index 442: Buenos Aires.
Retrieving Results for Index 483: Dolores.
Retrieving Results for Index 499: El Calvario.
Retrieving Results for Index 545: Porbandar.
Retrieving Results for Index 576: Jiwani.


,City,Country,Lat,Lng,Hotel Name
15,Mar del Plata,AR,-38.0023,-57.5575,
40,Hermanus,ZA,-34.4187,19.2345,
55,Castro,BR,-24.7911,-50.0119,
171,Champasak,LA,14.7500,106.0000,
258,Upleta,IN,21.7333,70.2833,
295,Vaijāpur,IN,19.9167,74.7333,
319,Coyhaique,CL,-45.5752,-72.0662,
339,Pathein,MM,16.7833,94.7333,
358,Awjilah,LY,29.1081,21.2869,
399,Ormara,PK,25.2088,64.6357,


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))